api doc : https://www.coingecko.com/api/documentations/v3

In [1]:
import numpy as np
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import time
import datetime
import pandas as pd
from pathlib import Path
import os

url = 'https://api.coingecko.com/api/v3/coins/bitcoin/history'
parameters = {
  'date':'12-11-2020'
}

session = Session()

url = 'https://api.coingecko.com/api/v3/coins/bitcoin/history'
parameters = {
  'date':'12-11-2020',
  'localization' : False
}

session = Session()


https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from=1604188800&to=1605283978
endpoint pour une fourchette de date

In [2]:
start_date = "24/11/2020"
end_date = "25/11/2020"
start_timestamp = int(time.mktime(datetime.datetime.strptime(start_date, "%d/%m/%Y").timetuple()))
end_timestamp  = int(time.mktime(datetime.datetime.strptime(end_date, "%d/%m/%Y").timetuple()))

print(start_timestamp, end_timestamp)

1606172400 1606258800


In [3]:
currency="bitcoin"
url = 'https://api.coingecko.com/api/v3/coins/'+currency+'/market_chart/range?'
parameters = {
    'vs_currency':'usd',
    'from':start_timestamp,
    'to':end_timestamp
}

session = Session()

try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    mypath = os.path.abspath('')
    path_to_save = str(mypath) + '/../data/'
    df = pd.DataFrame(data["prices"])
    file_name = "data.csv"
    df.to_csv(os.path.join(path_to_save, file_name))
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [4]:
csv_file_name = '../data/data.csv' #please put the name of the csv file you want to read
data = pd.read_csv(csv_file_name, index_col=[0],sep=',', encoding="utf-8")
data.iloc[:49] # we display the first 50 rows

,0,1
0,1606173556811,18316.102066
1,1606177044644,18273.492427
2,1606181004181,18450.612950
3,1606184361569,18390.498946
4,1606187723077,18332.293717
5,1606191561440,18349.033019
6,1606194969792,18374.112234
7,1606198029568,18303.951253
8,1606201977184,18407.624007
9,1606205144691,18369.329819


In [5]:
data.columns = ['time', 'price']

In [6]:
data

,time,price
0,1606173556811,18316.102066
1,1606177044644,18273.492427
2,1606181004181,18450.612950
3,1606184361569,18390.498946
4,1606187723077,18332.293717
5,1606191561440,18349.033019
6,1606194969792,18374.112234
7,1606198029568,18303.951253
8,1606201977184,18407.624007
9,1606205144691,18369.329819


In [21]:
timestamp = "1648753215"
newtimestamp = timestamp.substring(0, str.length - 3)

AttributeError: 'str' object has no attribute 'substring'

In [23]:
data['time'][0]

'0     1606173556811\n1     1606177044644\n2     1606181004181\n3     1606184361569\n4     1606187723077\n5     1606191561440\n6     1606194969792\n7     1606198029568\n8     1606201977184\n9     1606205144691\n10    1606209291978\n11    1606213246424\n12    1606216396325\n13    1606220202069\n14    1606223750298\n15    1606226691681\n16    1606230773573\n17    1606234417911\n18    1606237715845\n19    1606241189810\n20    1606245256931\n21    1606249215614\n22    1606252332722\n23    1606255553315\nName: time, dtype: in'

In [18]:
int(data['time'][0]).substring(0, str.length - 3)

ValueError: invalid literal for int() with base 10: '0     1606173556811\n1     1606177044644\n2     1606181004181\n3     1606184361569\n4     1606187723077\n5     1606191561440\n6     1606194969792\n7     1606198029568\n8     1606201977184\n9     1606

In [16]:
str(data["time"])

'0     0     1606173556811\\n1     1606177044644\\n2   ...\n1     0     1606173556811\\n1     1606177044644\\n2   ...\n2     0     1606173556811\\n1     1606177044644\\n2   ...\n3     0     1606173556811\\n1     1606177044644\\n2   ...\n4     0     1606173556811\\n1     1606177044644\\n2   ...\n5     0     1606173556811\\n1     1606177044644\\n2   ...\n6     0     1606173556811\\n1     1606177044644\\n2   ...\n7     0     1606173556811\\n1     1606177044644\\n2   ...\n8     0     1606173556811\\n1     1606177044644\\n2   ...\n9     0     1606173556811\\n1     1606177044644\\n2   ...\n10    0     1606173556811\\n1     1606177044644\\n2   ...\n11    0     1606173556811\\n1     1606177044644\\n2   ...\n12    0     1606173556811\\n1     1606177044644\\n2   ...\n13    0     1606173556811\\n1     1606177044644\\n2   ...\n14    0     1606173556811\\n1     1606177044644\\n2   ...\n15    0     1606173556811\\n1     1606177044644\\n2   ...\n16    0     1606173556811\\n1     1606177044644\\n2   .

In [15]:
data["time"] = str(data["time"])[:len(str(data["time"]))-3]

In [40]:
data

,time,price
0,1970-01-01 00:26:46.173556811,18316.102066
1,1970-01-01 00:26:46.177044644,18273.492427
2,1970-01-01 00:26:46.181004181,18450.612950
3,1970-01-01 00:26:46.184361569,18390.498946
4,1970-01-01 00:26:46.187723077,18332.293717
5,1970-01-01 00:26:46.191561440,18349.033019
6,1970-01-01 00:26:46.194969792,18374.112234
7,1970-01-01 00:26:46.198029568,18303.951253
8,1970-01-01 00:26:46.201977184,18407.624007
9,1970-01-01 00:26:46.205144691,18369.329819


In [21]:
pd.to_datetime(1606173556811)

Timestamp('1970-01-01 00:26:46.173556811')

In [19]:
pd.to_datetime(1490195805, unit='s')

Timestamp('2017-03-22 15:16:45')

In [18]:
timestamp = 1545730073
datetime.fromtimestamp(timestamp)

datetime.datetime(2018, 12, 25, 10, 27, 53)

In [22]:
datetime.fromtimestamp(1606173556811)

ValueError: year 52867 is out of range

In [41]:
1606173556811[1:2]

<>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<ipython-input-41-d826b88b4a0b>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  1606173556811[1:2]


TypeError: 'int' object is not subscriptable

In [42]:
abs(1606173556811) % 100 

11

In [45]:
num = 1606173556811
int(str(num)[:len(str(num))-3])


1606173556